In [1]:
from peft import PeftConfig

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [2]:
#불러오기
FINETUNED_MODEL = "C:/Users/minkyu/Desktop/dacon accident prevention/model_weight/v1"

peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [8]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # 4비트 로드 활성화
    bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
    bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
)

In [12]:
# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:31<00:00,  4.47s/it]


In [13]:
# QLoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)

In [14]:
# QLoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\peft\tuners\lora\bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [16]:
train = pd.read_csv("C:/Users/minkyu/Desktop/open/train.csv", encoding = 'utf-8-sig')

In [17]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

In [18]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [19]:
from datasets import Dataset

# combined_training_data가 Pandas DataFrame인 경우
dataset_hf = Dataset.from_pandas(combined_training_data.reset_index(drop=True))


In [20]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.
- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.
- 특수문자를 포함하지 마세요.

{context}

### 질문:
{question}

### 답변:
"""

In [21]:
def generate_prompt(data_point):
    # 추가 context가 필요한 경우 여기서 설정 (현재는 빈 문자열)
    context = ""
    question = data_point["question"]
    # prompt_template을 사용해 prompt를 생성합니다.
    prompt = prompt_template.format(context=context, question=question)
    # 모델 입력으로 사용할 프롬프트와 정답(평가용)만 반환합니다.
    return {"text": prompt, "answer": data_point["answer"]}

# 기존 데이터셋의 모든 컬럼을 제거하고, generate_prompt에서 필요한 컬럼만 남깁니다.
remove_column_keys = list(dataset_hf.features.keys())
dataset_cvted = dataset_hf.shuffle(seed=42).map(generate_prompt, remove_columns=remove_column_keys)

Map: 100%|██████████| 23422/23422 [00:01<00:00, 13734.18 examples/s]


In [22]:
# 토크나이징 완료된 데이터셋을 train/test로 분리
split_datasets = dataset_cvted.train_test_split(test_size=0.1, seed=42)

train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]  # 보통 검증 용도로 사용

In [27]:
eval_dataset[0]['text']

"\n### 지침: 당신은 건설 안전 전문가입니다.\n질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.\n- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.\n- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.\n- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.\n- 특수문자를 포함하지 마세요.\n\n\n\n### 질문:\n작업 프로세스는 '연결작업'이며 떨어짐(2미터 이상 ~ 3미터 미만)발생, 사고 원인은 '시스템 동바리 설치작업간 안전벨트 고리 미체결'입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?\n\n### 답변:\n"

In [48]:
prompt = eval_dataset[25]['text']
# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=64)
outputs = pipe(
    prompt,
    do_sample=False,
    #temperature=0.1,
    #top_k=0,
    #top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True 
)
print(outputs[0]["generated_text"][len(prompt):])

Device set to use cuda:0


동결된 토사와 이물질 철저 점검 시스템 도입과 함께 작업자들 대상 정기적 교육 실시로 인식 제고 강화하고 즉시 보고 체계 구축하여 신속 대응 능력 향상시킵니다._ANTI_FREEZE 장비 활용도 확대합니다_.


In [47]:
eval_dataset[25]

{'answer': '자재 적재 시 적합한 장소에 보관과 파일인양 작업 전 이물질 제거 철저.',
 'text': "\n### 지침: 당신은 건설 안전 전문가입니다.\n질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.\n- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.\n- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.\n- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.\n- 특수문자를 포함하지 마세요.\n\n\n\n### 질문:\n작업 프로세스는 '항타 및 항발작업'이며 물체에 맞음발생, 사고 원인은 '강관말뚝에 붙어있던 동결된 토사의 제거 미흡(이물질 제거 미흡)'입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?\n\n### 답변:\n"}